# Logs analysis - Features engineering

# Setup

In [2]:
# Data wrangling - Spark
from pyspark.sql.functions import sha2, udf, col, split, monotonically_increasing_id, regexp_extract
from pyspark.sql.functions import max as Fmax
from pyspark.sql.functions import min as Fmin
from pyspark.sql.types import *
from pyspark.sql import Window

# Parsing  user agents
import user_agents

# Sys
import os
from pathlib import Path
import re

%matplotlib inline

In [8]:
ROOT_PATH = Path("..")
file_path = ROOT_PATH /"data"/"processed"/"cleaned_logs"

In [9]:
spark

In [10]:
basic_features = spark.read.parquet(file_path.__str__())

In [11]:
basic_features.printSchema()
n,p = basic_features.count(), len(basic_features.columns)
print(f"Dataset size: {n} rows and {p} column")

root
 |-- ip_address: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status_code: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- referer: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- datetime_utc: timestamp (nullable = true)

Dataset size: 5015806 rows and 7 column


In [6]:
sample = sc.parallelize(basic_features.take(5)).toDF().toPandas()
sample

,ip_address,request,status_code,size,referer,user_agent,datetime_utc
0,5.113.18.208,GET /apache-log/access.log HTTP/1.1,206,34792,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,2018-05-28 04:58:06
1,5.113.18.208,GET /apache-log/access.log HTTP/1.1,206,26184,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,2018-05-28 04:58:06
2,5.113.18.208,GET /apache-log/access.log HTTP/1.1,206,37384,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,2018-05-28 04:58:06
3,5.113.18.208,GET /apache-log/access.log HTTP/1.1,206,43192,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,2018-05-28 04:58:06
4,5.113.18.208,GET /apache-log/access.log HTTP/1.1,206,35984,http://www.almhuette-raith.at/apache-log/,Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:1...,2018-05-28 04:58:06


# Data annotation

Bot detection can be regarded as:

- As a supervised binary classfication problem. A prerequisite of a supervised approach is finding a labeled data set of server logs with a label `is_bot` as True or False 
- As a semi-supervised learning where some instances are labeld such as good bot in many cases and other are unlabeld.
- As unsupervised problem: the ML algorithm should figure out the suspect/anomalic behavior in the dataset based on input features only.

In this challenge, I focus mainly on supervised approaches using especially Rules from litterature research


# Detection rules

These rules are extracted from my research on the topic based on the following ressrouces:

- Research paper : Modeling a session-based bots' arrival processat a web server, by Suchacka et al. 2017

- Machine learning and Security, by Freeman and Chio 2018


The rules are divided into 2 categories: request based and aggregation based rules (either client based or session based).

- Request based detection ordered by ease of detection:
    - R1: Requesting the file robots.txt
    - R2: Bot name declared in user agent
    - R3: HTTP method HEAD
    - R4: Requests with error status code 4** or 5**
    - R5: Requests with curl and wget commands
    - R6: SQL injections


- Session based detection:
    - S1: Mean-time for page in session < 0.5s
    - S2: Empty referer of the first page in session
    - S3: All HTTP requests in a session are with empty referers
    - S4: Zero image to page ratio
    - S5: Sessions containing one request

# Feature engineering

To apply these rules, I need to extract more features from user agents and HTTP requests such as:

- 1 - Parsing HTTP requests:
    - HTTP method
    - URI
    - HTTP version
- 2 - Parsing user agents: client characteristics
    - browser
    - Device
    - OS
    
    
User agent contains a wealth of information about client browser. To parse user agents efficiently and exhaustively. I will use Python excellent library user_agents. For more details, please check this link: https://pypi.org/project/user-agents/ This library extract basic features such as device, OS and browser characteristics. user_agents is based on ua-parser is an open-source and community-driven, regexp-based user-agent parser. It has been ported to many different programming languages all of which share the same parser.

I checked that all bot names in the research article are included in the (long) list of user_agents library.

Some fo the advanced features include:
- is_mobile: whether user agent is identified as a mobile phone (iPhone, Android phones, Blackberry, Windows Phone devices etc)
- is_tablet: whether user agent is identified as a tablet device (iPad, Kindle Fire, Nexus 7 etc)
- is_pc: whether user agent is identified to be running a traditional “desktop” OS (Windows, OS X, Linux)
- is_touch_capable: whether user agent has touch capabilities
- is_bot: whether user agent is a search engine crawler/spider listed in large list of identified bots such as YottaaMonitor, BrowserMob, HttpMonitor, YandexBot or Slurp
    
- 3- Aggregating request based on:
    - `Client ID = user agent + IP address`

In [61]:
# user agent components schema
schema = StructType([StructField("browser_family", StringType(), False),
                StructField("browser_version_string", StringType(), False),
                StructField("os_family", StringType(), False),
                StructField("os_version_string", StringType(), False),
                StructField("device_family", StringType(), False),
                StructField("device_brand", StringType(), False),
                StructField("device_model", StringType(), False),
                StructField("is_mobile", BooleanType(), False),
                StructField("is_tablet", BooleanType(), False),
                StructField("is_touch_capable", BooleanType(), False),
                StructField("is_pc", BooleanType(), False),
                StructField("is_ua_bot", BooleanType(), False)])

def is_cmd(x):
    """
    Looking for terminal commands like curl and wget
    
    Arg:
    x, str, string to search in
    
    Return
    flag: int, is command in 
    """
    
    if x is None:
        flag = False
    else:
        m = re.findall("^(curl|wget)/",x)
        flag = len(m)>0
    
    return flag


def is_sql_cmd(x):
    """
    Looking for sql injection commands
    
    Arg:
    x, str, string to search in
    
    Return
    n_keys: int, the number of SQL key words
    """
    
    if x is None:
        n_keys = 0
    else:
        m = re.findall("(SELECT|UNION|ALL|FROM|AND)",x)
        n_keys = len(m)
    
    return n_keys 

def parse(ua):
    
    """
    Parse user agent based on Python lib user_agents parse method to extract
    valuable features
    
    Args:
    ua: str - raw user agent
    
    Returns:
    data: List: parsed user agent components
    """
        
    if ua is None:
        ua = ""
        
    _ua = user_agents.parse(ua)
    data = [_ua.browser.family,
             _ua.browser.version_string,
             _ua.os.family,
             _ua.os.version_string,
             _ua.device.family,
             _ua.device.brand,
             _ua.device.model,
             _ua.is_mobile,
             _ua.is_tablet,
             _ua.is_touch_capable,
             _ua.is_pc,
             _ua.is_bot]

    for i in range(len(data)):
        if data[i] is None:
            data[i] = 'Unknown'
    
    return data

def build_features(df):
    
    """
    Build features based on basic parsed logs to extract the 
    maximum of pertinent features.
    
    Args:
    df: Spark DataFrame - parsed logs
    
    Returns:
    features: Spark DataFrame: parsed logs
    """
    
    # Checking for robots.txt

    null_udf = udf(lambda x: None if x in ("","-",'"-"') else x)

    # Parsing HTTP requests
    features = df.\
                withColumn("request_parts", split(col("request"),"\s")).\
                withColumn("request_method", col("request_parts")[0]).\
                withColumn("request_uri", col("request_parts")[1]).\
                withColumn("http_version", col("request_parts")[2]).\
                withColumn("request_file", regexp_extract('request_uri', "/([^/]+)$", 1)).\
                withColumn("request_file", null_udf("request_file")).\
                fillna({'request_file': "<NOFILE>"}).\
                drop("request_parts").\
                persist()
    
    # Parsing user agents
    parse_ua_udf = udf(lambda x: parse(x), schema) 
    features = features.\
                    withColumn("parsed",parse_ua_udf("user_agent")).\
                    select('*',
                            col('parsed.browser_family').alias('browser_family'),
                            col('parsed.browser_version_string').alias('browser_version_string'),
                            col('parsed.os_family').alias('os_family'),
                            col('parsed.os_version_string').alias('os_version_string'),
                            col('parsed.device_family').alias('device_family'),
                            col('parsed.device_brand').alias('device_brand'),
                            col('parsed.device_model').alias('device_model'),
                            col('parsed.is_mobile').alias('is_mobile'),
                            col('parsed.is_tablet').alias('is_tablet'),
                            col('parsed.is_touch_capable').alias('is_touch_capable'),
                            col('parsed.is_pc').alias('is_pc'),
                            col('parsed.is_ua_bot').alias('is_ua_bot')).\
                    drop("parsed").\
                    persist()
    
    # Looking for CLI commands
    is_cmd_udf = udf(lambda x: is_cmd(x), BooleanType())
    features = features.\
                select("*", is_cmd_udf("user_agent").alias("is_cmd"))
    
    # Looking for SQL injections commands
    is_cmd_udf = udf(lambda x: is_sql_cmd(x), IntegerType())
    features = features.\
                select("*", is_cmd_udf("request").alias("is_sql_cmd"))
    
    return features


def index(df):
    
    """
    Add a request index and client index to perform aggregate bot detection
    
    Args:
    df: Spark DataFrame - featurized parsed logs
    
    Returns:
    indexed_features: Spark DataFrame: indexed logs features
    """


    udf_concat = udf(lambda x,y: x+"-"+y, StringType())


    indexed = df.\
                withColumn("request_id",monotonically_increasing_id()).\
                withColumn('ip_address_str', col("ip_address")).\
                withColumn('user_agent_str', col("user_agent")).\
                fillna({"ip_address_str":"<NULL>", "user_agent_str": "<NULL>"}).\
                withColumn('client', udf_concat("ip_address_str","user_agent_str")).\
                withColumn('client_hash', sha2("client",256).cast("string")).\
                drop("ip_address_str", "user_agent_str").\
                drop("client").\
                persist()
    
    window = Window.\
            partitionBy("client_hash").\
            orderBy("request_id").\
            rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

    indexed_features = indexed.\
                        withColumn("client_id", Fmax("request_id").over(window))

    return indexed_features

In [62]:
engineered_features = build_features(basic_features)
indexed_eng_features = index(engineered_features)

In [63]:
indexed_eng_features.printSchema()

root
 |-- ip_address: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status_code: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- referer: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- datetime_utc: timestamp (nullable = true)
 |-- request_method: string (nullable = true)
 |-- request_uri: string (nullable = true)
 |-- http_version: string (nullable = true)
 |-- request_file: string (nullable = false)
 |-- browser_family: string (nullable = true)
 |-- browser_version_string: string (nullable = true)
 |-- os_family: string (nullable = true)
 |-- os_version_string: string (nullable = true)
 |-- device_family: string (nullable = true)
 |-- device_brand: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- is_mobile: boolean (nullable = true)
 |-- is_tablet: boolean (nullable = true)
 |-- is_touch_capable: boolean (nullable = true)
 |-- is_pc: boolean (nullable = true)
 |-- is_ua_bot: boolean (nullable 

In [64]:
sample = sc.parallelize(indexed_eng_features.take(5)).toDF().toPandas()
sample.T

,0,1,2,3,4
ip_address,46.39.53.30,46.39.53.30,46.39.53.30,46.39.53.30,46.39.53.30
request,GET /administrator/ HTTP/1.1,GET /administrator/ HTTP/1.1,POST /administrator/index.php HTTP/1.1,GET /administrator/ HTTP/1.1,POST /administrator/index.php HTTP/1.1
status_code,200,200,200,200,200
size,4263,4263,4494,4263,4494
referer,None,None,http://almhuette-raith.at/administrator/,None,http://almhuette-raith.at/administrator/
user_agent,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...
datetime_utc,2016-02-18 09:02:32,2016-02-18 09:02:33,2016-02-18 09:02:33,2016-02-18 09:02:33,2016-02-18 09:02:33
request_method,GET,GET,POST,GET,POST
request_uri,/administrator/,/administrator/,/administrator/index.php,/administrator/,/administrator/index.php
http_version,HTTP/1.1,HTTP/1.1,HTTP/1.1,HTTP/1.1,HTTP/1.1


# Saving to parquet files

In [65]:
file_path = (ROOT_PATH / "data" / "processed" /"featurized_logs").__str__()
indexed_eng_features.write.mode("overwrite").parquet(file_path)